In [1]:
import nltk
import string
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
import scipy
from transformers import BertTokenizer
from multiprocessing import Pool, TimeoutError

2021-10-26 20:55:58.652466: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-26 20:55:58.652482: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# Read in the data
reviews = pd.read_csv('Reviews.csv')
reviews = reviews.sample(frac=0.2)

In [3]:
# Clean the data
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
reviews_without_stopwords = [word for word in reviews['Text'].str.split() if word not in stop_words]

In [4]:
punctuation = list(string.punctuation)

In [5]:
# Set everything to lower case and remove punctuation
reviews_without_stopwords_or_punctuation = []
# reviews_without_stopwords_or_punctuation_full_list = []
for st in reviews_without_stopwords:
	tmp = []
	for word in st:
		if word not in punctuation:
			tmp.append(word.lower())
			# reviews_without_stopwords_or_punctuation_full_list.append(word.lower())
	reviews_without_stopwords_or_punctuation.append(tmp)

In [6]:
# print(len(reviews_without_stopwords_or_punctuation))
print(reviews_without_stopwords_or_punctuation[0])
print(len(reviews_without_stopwords_or_punctuation))
# print(reviews['Score'])

['i', 'think', 'chai', 'tea', 'is', 'hard', 'to', 'find', 'with', 'a', 'smooth,', 'not', 'overly', 'done', 'flavor.', 'however,', 'leave', 'it', 'to', 'twinings', 'to', 'do', 'the', 'job', 'just', 'perfectly,', 'and', 'in', 'decaf!', 'i', 'purchased', 'this', 'tea', 'thinking,', '"here', 'i', 'go', 'again,"', 'hoping', 'for', 'the', 'perfect', 'cup', 'of', 'spice', 'and', 'tea.', 'this', 'one', 'fills', 'the', 'bill', 'and', 'then', 'some!', 'even', 'my', 'british', 'girlfriend', 'likes', 'this', 'one', '(hey,', 'twinings', 'is', 'from', 'london!).', "it's", 'not', 'over-spiced', 'like', 'so', 'many', "chai's", 'are', 'and', 'you', 'can', 'even', 'enjoy', 'this', 'one', 'without', 'sugar', 'and', 'milk', 'if', 'you', 'like', 'without', 'curling', 'your', 'hair', 'from', 'the', 'inside', 'out!', "it's", 'a', 'great', 'tea,', 'one', 'that', "i'm", 'going', 'to', 'be', 'buying', 'for', 'a', 'long', 'time', 'too', 'highly', 'recommend!']
113691


In [7]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()


def lemmatize_string(word_list):
	return ' '.join([lemmatizer.lemmatize(words) for words in word_list])

with Pool(processes=8) as pool:
	reviews_without_stopwords_or_punctuation_lemmatize = pool.map(lemmatize_string, reviews_without_stopwords_or_punctuation)

In [8]:
reviews_without_stopwords_or_punctuation_lemmatize[1]

"sometimes it's hard to find good quality product for the k-cups. this is a good tea. i love green tea and this is a good brand. i like this brand and bigelow organic green tea. i think the bigelow is my slight favorite, but this is good tea. no aftertaste and nor are there ground in the bottom of the cup. the price on this one put everything to shame. usually the k-cups are more expensive, so this is a bargain. get it now."

In [9]:
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import CountVectorizer
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(lowercase=True, stop_words=None,
                     ngram_range=(1, 1), tokenizer=token.tokenize)
reviews_tokenized = cv.fit_transform(reviews_without_stopwords_or_punctuation_lemmatize)

In [10]:
print(reviews_tokenized.shape)

(113691, 58823)


In [11]:
review_score = [int(i) for i in reviews['Score']]

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(reviews_tokenized, review_score, test_size=0.2, random_state=0)

In [13]:
# from sklearn.tree import DecisionTreeClassifier 
# from sklearn import metrics
# clf = DecisionTreeClassifier().fit(X_train, y_train)
# predicted= clf.predict(X_test)
# print("Decision Tree Accuracy:",metrics.accuracy_score(y_test, predicted))
# print("Precision: " + str(metrics.precision_score(y_test, predicted, average='macro')))
# print("Recall: " + str(metrics.recall_score(y_test, predicted, average='macro')))

In [14]:
# from sklearn.svm import SVC
# clf = SVC().fit(X_train, y_train)
# predicted= clf.predict(X_test)
# print("SVM Accuracy:",metrics.accuracy_score(y_test, predicted))
# print("Precision: " + str(metrics.precision_score(y_test, predicted, average='macro')))
# print("Recall: " + str(metrics.recall_score(y_test, predicted, average='macro')))

In [15]:
# Using bert
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [16]:
reviews_tokenized_bert = tokenizer.batch_encode_plus(reviews_without_stopwords_or_punctuation_lemmatize, max_length=512, pad_to_max_length=True, return_tensors='pt')

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/gabe/.local/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2211: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [17]:
print(reviews_tokenized_bert['input_ids'])

tensor([[ 101, 1045, 2228,  ...,    0,    0,    0],
        [ 101, 2823, 2009,  ...,    0,    0,    0],
        [ 101, 1045, 3984,  ...,    0,    0,    0],
        ...,
        [ 101, 2057, 2074,  ...,    0,    0,    0],
        [ 101, 2025, 4840,  ...,    0,    0,    0],
        [ 101, 2292, 2033,  ...,    0,    0,    0]])


In [18]:
print(reviews_tokenized_bert.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])


In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(reviews_tokenized_bert['input_ids'], review_score, test_size=0.2, random_state=0)

In [20]:
# from sklearn import metrics
# from sklearn.svm import SVC
# clf = SVC().fit(X_train, y_train)
# predicted= clf.predict(X_test)
# print("SVM Accuracy:",metrics.accuracy_score(y_test, predicted))
# print("Precision: " + str(metrics.precision_score(y_test, predicted, average='macro')))
# print("Recall: " + str(metrics.recall_score(y_test, predicted, average='macro')))

In [21]:
# Linear Support Vector Machine
from sklearn import metrics
from sklearn.svm import LinearSVC
clf = LinearSVC(max_iter=1000).fit(X_train, y_train)
predicted= clf.predict(X_test)
print("Linear Support Vector Machine Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Precision: " + str(metrics.precision_score(y_test, predicted, average='macro')))
print("Recall: " + str(metrics.recall_score(y_test, predicted, average='macro')))

Linear Support Vector Machine Accuracy: 0.4991864198073794
Precision: 0.21536380155123752
Recall: 0.21034048696583216


/home/gabe/.local/lib/python3.9/site-packages/sklearn/svm/_base.py:1199: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
